In [12]:
import pandas as pd
import numpy as np

In [13]:
# 
Password_Attack = pd.read_csv("Password_attack.csv")
print("Password_Attack loaded!")
SQL_injection = pd.read_csv("SQL_injection_attack.csv")
print("SQL_injection loaded!")
XSS_attack = pd.read_csv("XSS_attack.csv")
print("XSS_attack loaded!")
Uploading_attack = pd.read_csv("Uploading_attack.csv")
print("Uploading_attack loaded!")

C:\Users\ParkJaehyung\AppData\Local\Temp\ipykernel_11628\1384700295.py:2: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  Password_Attack = pd.read_csv("Password_attack.csv")


Password_Attack loaded!
SQL_injection loaded!
XSS_attack loaded!
Uploading_attack loaded!


In [25]:
common_columns = []
uncommon_columns = []
for col in set([*SQL_injection.columns, *XSS_attack.columns, *Uploading_attack.columns, *Password_Attack.columns]):
    if col in SQL_injection.columns and col in XSS_attack.columns and col in Uploading_attack.columns and col in Password_Attack.columns:
        common_columns.append(col)
    else: 
        uncommon_columns.append(col)

In [26]:
uncommon_columns

[]

In [27]:
common_columns

['tcp.connection.syn',
 'dns.qry.name.len',
 'http.request.uri.query',
 'ip.src_host',
 'dns.retransmit_request',
 'mqtt.msgtype',
 'frame.time',
 'tcp.flags.ack',
 'mqtt.topic_len',
 'icmp.seq_le',
 'arp.hw.size',
 'mqtt.msg',
 'dns.qry.qu',
 'icmp.unused',
 'http.tls_port',
 'dns.retransmit_request_in',
 'tcp.srcport',
 'mqtt.len',
 'tcp.checksum',
 'Attack_label',
 'mqtt.proto_len',
 'http.response',
 'Attack_type',
 'tcp.flags',
 'dns.qry.name',
 'dns.retransmission',
 'arp.dst.proto_ipv4',
 'arp.src.proto_ipv4',
 'tcp.connection.rst',
 'mbtcp.len',
 'mqtt.ver',
 'tcp.len',
 'ip.dst_host',
 'udp.port',
 'mqtt.conack.flags',
 'http.file_data',
 'mqtt.msg_decoded_as',
 'mbtcp.unit_id',
 'tcp.ack_raw',
 'icmp.transmit_timestamp',
 'tcp.options',
 'mqtt.protoname',
 'mqtt.conflags',
 'mqtt.topic',
 'tcp.payload',
 'arp.opcode',
 'icmp.checksum',
 'dns.qry.type',
 'tcp.connection.synack',
 'http.request.version',
 'mqtt.conflag.cleansess',
 'udp.time_delta',
 'tcp.ack',
 'tcp.dstport',


In [66]:
pd.concat([SQL_injection["tcp.connection.syn"], XSS_attack["tcp.connection.syn"]]).unique().shape[0] > 1

True

In [67]:
cols = list(Password_Attack.columns)
unused = []
for col in common_columns:
    series = pd.concat([SQL_injection[col], XSS_attack[col], Uploading_attack[col], Password_Attack[col]])
    unique_datas = series.unique()
    if unique_datas.shape[0] <= 1:
        cols.remove(col)
        unused.append(col)

In [68]:
cols, unused


(['frame.time',
  'ip.src_host',
  'ip.dst_host',
  'arp.dst.proto_ipv4',
  'arp.opcode',
  'arp.hw.size',
  'arp.src.proto_ipv4',
  'http.file_data',
  'http.content_length',
  'http.request.uri.query',
  'http.request.method',
  'http.referer',
  'http.request.full_uri',
  'http.request.version',
  'http.response',
  'tcp.ack',
  'tcp.ack_raw',
  'tcp.checksum',
  'tcp.connection.fin',
  'tcp.connection.rst',
  'tcp.connection.syn',
  'tcp.connection.synack',
  'tcp.dstport',
  'tcp.flags',
  'tcp.flags.ack',
  'tcp.len',
  'tcp.options',
  'tcp.payload',
  'tcp.seq',
  'tcp.srcport',
  'Attack_type'],
 ['dns.qry.name.len',
  'dns.retransmit_request',
  'mqtt.msgtype',
  'mqtt.topic_len',
  'icmp.seq_le',
  'mqtt.msg',
  'dns.qry.qu',
  'icmp.unused',
  'http.tls_port',
  'dns.retransmit_request_in',
  'mqtt.len',
  'Attack_label',
  'mqtt.proto_len',
  'dns.qry.name',
  'dns.retransmission',
  'mbtcp.len',
  'mqtt.ver',
  'udp.port',
  'mqtt.conack.flags',
  'mqtt.msg_decoded_as',
 

In [69]:
import json
with open("column_data.txt", "w") as f:
    f.write(json.dumps({"Used_Columns":cols, "Unused_Columns":unused}, indent=4))
    # f.write("Used Columns\n")
    # f.write(",".join(cols)+"\n")
    # f.write("Unused Columns\n")
    # f.write(",".join(unused)+"\n")

In [70]:
concat_datas = pd.DataFrame({col:pd.concat([SQL_injection[col], XSS_attack[col], Uploading_attack[col], Password_Attack[col]]) for col in cols})

In [86]:
processed_data = concat_datas

In [97]:
with open("unique_values.txt", "w") as f:
    for col in cols:
        count = processed_data[col].unique().shape[0]
        unique_cols = list(processed_data[col].unique() if count < 10 else processed_data[col].unique()
        f.write(f"{col=} | count={count} \n\t\tunique vals =  {unique_cols}\n\n")

In [85]:
## removed the data of No-Method 
# temp = processed_data.loc[processed_data["http.request.method"] != '0']
processed_data["http.request.method"].unique()

array(['0', 'GET', 'TRACE', 'POST'], dtype=object)

In [88]:
processed_data["arp.dst.proto_ipv4"].unique()

array(['192.168.0.1', '192.168.0.128', '0', '192.168.0.170',
       '192.168.0.152', 0], dtype=object)

In [87]:
processed_data.to_csv("Concat_Processed_Data.csv", sep=",")